# Youtube statistics  Readme

This program leverages MySQL and Python to provide users with a self-exploratory user interface for extracting data insights and patterns. Developed by students, it acknowledges certain limitations and optimizes the original database for improved usability and handling within the code.

The database comprises five tables, four of which categorize data by country (United States, India, Canada, and Great Britain), while the fifth table, the Category table, consolidates category information. The Category table was manually created by the students to ensure consistency across category IDs and names, which were originally scattered across the other tables in the database. Initially, the database included a broader range of columns, but to streamline the dataset for better analysis, certain columns were removed.

The original database schema included fields such as video_id, trending_date, title, channel, category, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, and video_error_or_removed. However, through iterative refinement, the dataset was optimized to include only the most relevant attributes:
video_id (varchar),
trending_date (date),
title (varchar),
channel (varchar),
category (int),
publish_time (time),
views (bigint),
likes (bigint),
dislikes (bigint),
comment_count (int),
comments_disabled (bool),
ratings_disabled (bool),
video_error_or_removed (bool).

Queries were used to extract essential aspects of the dataset, resulting in a more concise and manageable dataset. We hope users find the program intuitive and valuable for their data exploration needs.

Authors – Arunima Srikant and Ayesha Rao

In [1]:
import tkinter as tk
from tkinter import ttk
import mysql.connector
import re

def show_statistics():
    selected_region = combobox_region.get()
    
    if selected_region == "India":
        popup_title = "Indian Statistics"
        table_name = "india"
    elif selected_region == "Canada":
        popup_title = "Canadian Statistics"
        table_name = "canada"
    elif selected_region == "USA":
        popup_title = "USA Statistics"
        table_name = "united_states"
    elif selected_region == "Great Britain":
        popup_title = "Great Britain Statistics"
        table_name = "great_britain"
    else:
        popup_title = "Invalid Selection"
        popup_message = "Please select a valid region."

    popup_window = tk.Toplevel(root)
    popup_window.title(popup_title)
    
    # Labels
    filter_label = ttk.Label(popup_window, text="Filter:")
    filter_label.grid(row=0, column=0, padx=5, pady=5)
    
    # Combo Boxes
    filter_combobox_1 = ttk.Combobox(popup_window, values=[
        "trending_date", "category_id", "views", "likes", "dislikes", "comment_count"
    ])
    filter_combobox_1.grid(row=1, column=0, padx=5, pady=5)

    # Explore Button
    explore_button_popup = ttk.Button(popup_window, text="Explore", command=lambda: explore_selected_filter(filter_combobox_1.get(), table_name))
    explore_button_popup.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def explore_selected_filter(selected_filter, table_name):
    if selected_filter == "trending_date":
        show_trending_date_popup(table_name)
    elif selected_filter == "category_id":
        show_category_popup(table_name)
    elif selected_filter in ['views', 'likes', 'dislikes', 'comment_count']:
        show_numeric_filter_popup(selected_filter, table_name)
    else:
        popup_window = tk.Toplevel(root)
        popup_window.title("Selected Filter")
        popup_message = f"The filter you chose is '{selected_filter}'"
        popup_label = ttk.Label(popup_window, text=popup_message)
        popup_label.pack(padx=10, pady=10)

def show_trending_date_popup(table_name):
    popup = tk.Toplevel(root)
    popup.title("Enter Trending Date")
    popup.geometry("300x150")
    
    # Labels and Entry for entering date and comparison operator
    comparison_label = ttk.Label(popup, text="Comparison Operator:")
    comparison_label.grid(row=0, column=0, padx=5, pady=5)
    comparison_entry = ttk.Entry(popup)
    comparison_entry.grid(row=0, column=1, padx=5, pady=5)
    
    date_label = ttk.Label(popup, text="Enter date:")
    date_label.grid(row=1, column=0, padx=5, pady=5)
    date_entry = ttk.Entry(popup)
    date_entry.grid(row=1, column=1, padx=5, pady=5)
    
    # Button to save the entered data
    save_button = ttk.Button(popup, text="Save", command=lambda: save_date(popup, comparison_entry.get(), date_entry.get(), table_name))
    save_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def save_date(popup, comparison, date, table_name):
    global filter_var
    if not re.match(r"([><]=?)?\d{4}-\d{2}-\d{2}", date):
        show_error_popup(popup, "Please enter the date in the correct format (yyyy-mm-dd).")
        return
    
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()
    
    # Execute SQL query
    cursor.execute(f"SELECT {table_name}.title, {table_name}.channel, category.category_name, {table_name}.publish_time, {table_name}.trending_date, {table_name}.views, {table_name}.likes, {table_name}.dislikes, {table_name}.comment_count FROM category,{table_name} WHERE category.category_id = {table_name}.category AND trending_date {comparison} '{date}'")
    result = cursor.fetchall()
    
    # Display the result
    show_treeview(table_name, result)
    
    cursor.close()
    connection.close()

    popup.destroy()

# Similar modifications for other save functions

def show_category_popup(table_name):
    popup = tk.Toplevel(root)
    popup.title("Choose Category")
    
    # Radio buttons for choosing category
    categories = {
        1: "Film and Animation",
        2: "Autos and Vehicles",
        10: "Music",
        15: "Pets & Animals",
        17: "Sports",
        18: "Short Movies",
        19: "Travel & Events",
        20: "Gaming",
        21: "Videoblogging",
        22: "People & Blogs",
        23: "Comedy",
        24: "Entertainment",
        25: "News & Politics",
        26: "Howto & Style",
        27: "Education",
        28: "Science & Technology",
        30: "Movies",
        31: "Anime/Animation",
        32: "Action/Adventure",
        33: "Classics",
        35: "Documentary",
        36: "Drama",
        37: "Family",
        38: "Foreign",
        39: "Horror",
        40: "Sci-Fi/Fantasy",
        41: "Thriller",
        42: "Shorts",
        43: "Shows",
        44: "Trailers"
    }
    
    # Determine number of rows for radio buttons
    num_rows = (len(categories) + 1) // 2
    
    # Initialize row and column counters
    row = 0
    col = 0
    
    for cat_id, cat_name in categories.items():
        radio = ttk.Radiobutton(popup, text=cat_name, variable=category_var, value=cat_id)
        radio.grid(row=row, column=col, sticky=tk.W, padx=5, pady=2)
        
        # Increment row and column counters
        if row < num_rows - 1:
            row += 1
        else:
            row = 0
            col += 1
    
    # Button to save the selected category
    save_button = ttk.Button(popup, text="Save", command=lambda: save_category(popup, table_name))
    save_button.grid(row=num_rows, column=0, columnspan=2, padx=5, pady=5)

def save_category(popup, table_name):
    category_chosen = category_var.get()
    
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()
    
    # Execute SQL query with JOIN to filter by category_id
    cursor.execute(f"SELECT {table_name}.title, {table_name}.channel, category.category_name, {table_name}.publish_time, {table_name}.trending_date, {table_name}.views, {table_name}.likes, {table_name}.dislikes, {table_name}.comment_count FROM {table_name} INNER JOIN category ON {table_name}.category = category.category_id WHERE category.category_id = {category_chosen}")
    result = cursor.fetchall()
    
    # Display the result
    show_treeview(table_name, result)
    
    cursor.close()
    connection.close()
    
    popup.destroy()

# Similar modifications for other functions

def show_treeview(selected_region, data):
    popup = tk.Toplevel(root)
    popup.title(f"{selected_region} Statistics")
    
    tree_frame = ttk.Frame(popup)
    tree_frame.pack(fill=tk.BOTH, expand=True)

    tree = ttk.Treeview(tree_frame)
    tree["columns"] = ("one", "two", "three", "four", "five", "six", "seven", "eight", "nine")
    
    tree.heading("#0", text="Title")
    tree.heading("one", text="Channel Title")
    tree.heading("two", text="Category Name")
    tree.heading("three", text="Publish Time")
    tree.heading("four", text="Trending Date")
    tree.heading("five", text="Views")
    tree.heading("six", text="Likes")
    tree.heading("seven", text="Dislikes")
    tree.heading("eight", text="Comment Count")
    
    # Configure vertical scrollbar
    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
    vsb.pack(side=tk.RIGHT, fill=tk.Y)
    tree.configure(yscrollcommand=vsb.set)

    # Configure horizontal scrollbar
    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=tree.xview)
    hsb.pack(side=tk.BOTTOM, fill=tk.X)
    tree.configure(xscrollcommand=hsb.set)

    for row in data:
        tree.insert("", "end", text=row[0], values=row[1:])
    
    tree.pack(fill=tk.BOTH, expand=True)


def show_numeric_filter_popup(selected_filter, table_name):
    popup = tk.Toplevel(root)
    popup.title(f"Filter by {selected_filter.capitalize()}")

    # Label, Combo Box for comparison operator, and Entry for entering the number
    comparison_label = ttk.Label(popup, text="Comparison Operator:")
    comparison_label.grid(row=0, column=0, padx=5, pady=5)

    comparison_combobox = ttk.Combobox(popup, values=['<', '>', '='])
    comparison_combobox.grid(row=0, column=1, padx=5, pady=5)

    number_label = ttk.Label(popup, text=f"Enter the number of {selected_filter}:")
    number_label.grid(row=1, column=0, padx=5, pady=5)

    number_entry = ttk.Entry(popup)
    number_entry.grid(row=1, column=1, padx=5, pady=5)

    # Button to apply the filter
    apply_button = ttk.Button(popup, text="Apply Filter", command=lambda: apply_numeric_filter(popup, selected_filter, comparison_combobox.get(), number_entry.get(), table_name))
    apply_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def apply_numeric_filter(popup, selected_filter, comparison_operator, number, table_name):
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()

    # Execute SQL query with numeric filter
    cursor.execute(f"SELECT {table_name}.title, {table_name}.channel, category.category_name, {table_name}.publish_time, {table_name}.trending_date, {table_name}.views, {table_name}.likes, {table_name}.dislikes, {table_name}.comment_count FROM {table_name} INNER JOIN category ON {table_name}.category = category.category_id WHERE {table_name}.{selected_filter} {comparison_operator} {number}")
    result = cursor.fetchall()

    # Display the result
    show_treeview(table_name, result)

    cursor.close()
    connection.close()

    popup.destroy()

root = tk.Tk()
root.geometry("700x100+400+300")
root.title("YouTube Statistics")

# Create main window widgets
description = ttk.Label(root, text="Description:")
explanation = ttk.Label(root, text = "This is a self-exploratory interface that searches through trending YouTube statistics of four regions. Choose a region to continue.")

# Populate the combobox with Genre names
combobox_region = ttk.Combobox(root, values=['India', 'Canada', 'USA', 'Great Britain'])

# Create an explore button
explore_button = ttk.Button(root, text="Explore", command=show_statistics)

# Place widgets on the main window
description.pack()
explanation.pack()
combobox_region.pack()
explore_button.pack()

# Initialize category variable
category_var = tk.IntVar()

root.mainloop()
